In [40]:
import pandas as pd
import numpy as np
import json

In [41]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# thank you to @kinghelix and @trevormarburger for this idea
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

# Simple test examples
if __name__ == '__main__':
    print("Wisconin --> WI?", us_state_abbrev['Wisconsin'] == 'WI')
    print("WI --> Wisconin?", abbrev_us_state['WI'] == 'Wisconsin')
    print("Number of entries (50 states, DC, 5 Territories) == 56? ", 56 == len(us_state_abbrev))


Wisconin --> WI? True
WI --> Wisconin? True
Number of entries (50 states, DC, 5 Territories) == 56?  True


In [42]:
def extract_normalized_state_dfs(df, date_idx):
    """Normalize each state data by using the same date range."""
    dfs = []
    for group in df.groupby(["state_code"]):
        state_df = (
            group[1][["total_cases", "total_deaths"]]
            .reindex(date_idx, method="pad")
            .fillna(0)
        )
        state_df["total_cases"] = state_df["total_cases"].astype("int64")
        state_df["total_deaths"] = state_df["total_deaths"].astype("int64")
        state_df.loc[:, "state_code"] = group[0]
        state_df.loc[:, "state_name"] = group[1]["state_name"][-1]
        state_df.loc[:, "fips"] = group[1]["fips"][-1]
        dfs.append(
            state_df[
                ["state_code", "fips", "state_name", "total_cases", "total_deaths"]
            ]
        )
    return dfs

In [43]:
df = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv", index_col = 0,
                 parse_dates = True).rename(
    columns = {'cases': 'total_cases', 'deaths': 'total_deaths', 'state':'state_name'}
)

In [44]:
df.head()

,state_name,fips,total_cases,total_deaths
date,,,,
2020-01-21,Washington,53,1,0
2020-01-22,Washington,53,1,0
2020-01-23,Washington,53,1,0
2020-01-24,Illinois,17,1,0
2020-01-24,Washington,53,1,0


In [45]:
df['state_code'] = df.state_name.apply(lambda x: us_state_abbrev[x])

In [46]:
df.sample(10)

,state_name,fips,total_cases,total_deaths,state_code
date,,,,,
2020-04-04,Virginia,51,2408,52,VA
2020-03-01,Texas,48,11,0,TX
2020-08-19,Virginia,51,109019,2410,VA
2020-09-26,New York,36,459472,32713,NY
2020-04-06,New Mexico,35,686,12,NM
2020-08-02,Delaware,10,14949,585,DE
2020-04-01,Puerto Rico,72,286,11,PR
2020-04-09,Illinois,17,16422,534,IL
2020-09-25,Guam,66,3255,40,GU


In [47]:
df['state_code'].isnull().values.any()

False

In [48]:
idx = pd.date_range('2019-12-31', df.index.max())

In [49]:
df.groupby(["state_code"]).count().head()

,state_name,fips,total_cases,total_deaths
state_code,,,,
AK,224,224,224,224
AL,223,223,223,223
AR,225,225,225,225
AZ,270,270,270,270
CA,271,271,271,271


In [50]:
NYT_normalized = pd.concat(extract_normalized_state_dfs(df, idx)).reset_index().rename(
    columns={"index": "date"}
)

In [53]:
NYT_normalized.head()

,date,state_code,fips,state_name,total_cases,total_deaths
0,2019-12-31,AK,2,Alaska,0,0
1,2020-01-01,AK,2,Alaska,0,0
2,2020-01-02,AK,2,Alaska,0,0
3,2020-01-03,AK,2,Alaska,0,0
4,2020-01-04,AK,2,Alaska,0,0


In [55]:
NYT_normalized.to_csv('outputs/cases_us_states_nyt.csv', index=False)

In [52]:
def prepare_data_structure(df, gby="state_code"):
    data = []
    for g in df.groupby([gby]):
        code = g[0]
        temp_df = g[1]
        try:
            country_data = {
                "code": code,
                "name": temp_df["state_name"].iloc[0],
                "confirmed": list(zip(temp_df.date, temp_df.total_cases)),
                "deaths": list(zip(temp_df.date, temp_df.total_deaths)),
            }
            data.append(country_data)
        except KeyError:
            print("metadata doesn't exist for: ", code)
            continue
    return data